# Toxic Comment Classification Final Project

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from google.colab import drive
# import nltk
import re
# nltk.download('popular')
drive.mount('/content/gdrive')

import itertools
import math
import pickle
import statistics
import pdb

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/Final Project/data
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
test_labels = pd.read_csv('test_labels.csv')
test_labels = test_labels.drop(columns= 'id')

print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

import random
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

/content/gdrive/My Drive/Colab Notebooks/Final Project/data
True
Using device: cuda


In [ ]:
merged = pd.concat([test, test_labels], axis = 1)
# merged = merged.replace(-1, 0)
# df = df[df.line_race != 0]
merged = merged[merged.identity_hate != -1]
data = pd.concat([train, merged])
data = data.drop(columns= 'id')

cols = data.columns[1:]
# data["not_toxic"] = np.where(data[cols].eq(1).any(1), 0, 1)

In [ ]:
data

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...
153150,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
153151,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
153154,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
153155,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [ ]:
text_all = data.drop(columns= ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', ])
labels_all = data.drop(columns= 'comment_text')

text_all = text_all.values.tolist()
text_all = [item for sublist in text_all for item in sublist]
labels_all = labels_all.values.tolist()

temp = list(zip(text_all, labels_all))
random.shuffle(temp)
text_all, labels_all = zip(*temp)

In [ ]:
text = text_all[:-70000]
labels = labels_all[:-70000]

text_val = text_all[-70000:]
labels_val = labels_all[-70000:]

In [ ]:
pad_word = "<pad>"
bos_word = "<s>"
eos_word = "</s>"
unk_word = "<unk>"
pad_id = 0
unk_id = 1
def normalize_sentence(s):
    s = s.lower()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

class Vocabulary:
    def __init__(self):
        self.word_to_id = {pad_word: pad_id, unk_word: unk_id}
        self.word_count = {}
        self.id_to_word = {pad_id: pad_word,unk_id: unk_word}
        self.num_words = 2

        self.char_to_id = {pad_word: pad_id, unk_word: unk_id}
        self.char_count = {}
        self.id_to_char = {pad_id: pad_word, unk_id: unk_word}
        self.num_char = 2
    
    def get_word_ids_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        sent_ids = [self.word_to_id[word] if word in self.word_to_id \
                               else unk_id for word in sentence.split()]
        return sent_ids
    
    def get_char_ids_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        sent_ids = [self.char_to_id[char] if char in self.char_to_id \
                               else unk_id for char in list(sentence)] 
        return sent_ids
    
    def word_tokenized_sentence(self, sentence):
        sent_ids = self.get_word_ids_from_sentence(sentence)
        return [self.id_to_word[word_id] for word_id in sent_ids]
    def char_tokenized_sentence(self, sentence):
        char_ids = self.get_char_ids_from_sentence(sentence)
        return [self.id_to_char[char_id] for char_id in char_ids]

    def decode_sentence_from_word_ids(self, sent_ids):
        words = list()
        for i, word_id in enumerate(sent_ids):
            if word_id in [pad_id]:
                # Skip these words
                continue
            else:
                words.append(self.id_to_word[word_id])
        return ' '.join(words)
    def decode_sentence_from_char_ids(self, sent_ids):
        words = list()
        for i, word_id in enumerate(sent_ids):
            if word_id in [ pad_id]:
                # Skip these words
                continue
            else:
                words.append(self.id_to_char[word_id])
        return ''.join(words)

    def add_words_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        for word in sentence.split():
            if word not in self.word_to_id:
                # add this word to the vocabulary
                self.word_to_id[word] = self.num_words
                self.id_to_word[self.num_words] = word
                self.word_count[word] = 1
                self.num_words += 1
            else:
                # update the word count
                self.word_count[word] += 1
    def add_chars_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        for word in list(sentence):
            if word not in self.char_to_id:
                # add this word to the vocabulary
                self.char_to_id[word] = self.num_char
                self.id_to_char[self.num_char] = word
                self.char_count[word] = 1
                self.num_char += 1
            else:
                # update the word count
                self.char_count[word] += 1

In [ ]:
vocab_init = Vocabulary()
for src in text:
    vocab_init.add_words_from_sentence(src)
    vocab_init.add_chars_from_sentence(src)

In [ ]:
word_tokens_sorted = sorted(vocab_init.word_count.items(), key=lambda item: item[1], reverse=True)
char_tokens_sorted = sorted(vocab_init.char_count.items(), key=lambda item: item[1], reverse=True)

In [ ]:
vocab = Vocabulary()
vocab_limit = 20000
for word_token in word_tokens_sorted[:vocab_limit]:
  vocab.add_words_from_sentence(word_token[0])

for char_token in char_tokens_sorted:
  vocab.add_chars_from_sentence(char_token[0])

In [ ]:
print("Length of train corpus:", len(text))
print("Number of word tokens:", vocab.num_words)
print("Number of char tokens:", vocab.num_char)

sample_text = text[13]
print(sample_text)
print(vocab.get_word_ids_from_sentence(sample_text)[0:10])
print(vocab.get_char_ids_from_sentence(sample_text)[0:10])

Length of train corpus: 153549
Number of word tokens: 20002
Number of char tokens: 31
This article should simply provide readers of a clearly defined definition of extended warranties, not whether or not they are worthwhile.  That subject is purely based upon opinion.  Some people believe in extended warranties, and I can tell you for a fact that Circuit City makes 60% profit on laptop protection plans, but that DOESN'T make them worthless.  It can be beneficial to both parties involved.  That's how business works, right? This really should be changed.
[16, 28, 63, 293, 475, 940, 6, 8, 374, 1735]
[3, 10, 5, 8, 1, 4, 9, 3, 5, 14]


In [ ]:
longest_sentence_length = -1
longest_sentence_index = -1

for i in range(len(text)):
  if len(normalize_sentence(text[i])) > longest_sentence_length:
    longest_sentence_length = len(normalize_sentence(text[i]))
    longest_sentence_index = i

In [ ]:
print(longest_sentence_length)
print(longest_sentence_index)
print(len(text), len(labels))
print(text[longest_sentence_index][0:100])
print(labels[longest_sentence_index])

9909
65707
153549 153549
== school sucks == 

 school sucks!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
[1, 0, 1, 0, 0, 0]


In [ ]:
class ToxicDataset(Dataset):

    def __init__(self, text, labels, vocab, max_slen, max_clen, device):
        """
        Args:
            conversations: list of tuple (src_string, tgt_string) 
                         - src_string: String of the source sentence
                         - tgt_string: String of the target sentence
            vocab: Vocabulary object that contains the mapping of 
                    words to indices
            device: cpu or cuda
        """
        self.text = text
        self.labels = labels
        self.vocab = vocab
        self.device = device

        def encode_word(src):
          keep_len = min(len(self.vocab.get_word_ids_from_sentence(src)), max_slen)
          return self.vocab.get_word_ids_from_sentence(src)[:keep_len]
        def encode_char(src):
          keep_len = min(len(self.vocab.get_char_ids_from_sentence(src)), max_clen)
          return self.vocab.get_char_ids_from_sentence(src)[:keep_len]

        # We will pre-tokenize the conversations and save in id lists for later use
        self.word_conversations = [encode_word(src) for src in self.text]
        self.char_conversations = [encode_char(src) for src in self.text]
        
    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return {"word_ids":self.word_conversations[idx],\
                "char_ids": self.char_conversations[idx],\
                "text": self.text[idx],\
                "labels":self.labels[idx]}
        # return {"conv_ids":self.tokenized_conversations[idx], "conv":self.conversations[idx]}

In [ ]:
def collate_fn(data):
    # Sort conv_ids based on decreasing order of the src_lengths.
    # This is required for efficient GPU computations.
    word_ids = [torch.LongTensor(e["word_ids"]) for e in data]
    src_str_word = [e["text"] for e in data]
    word_labels = [torch.LongTensor(e["labels"]) for e in data]
    word_data = list(zip(word_ids, word_labels, src_str_word))
    word_data.sort(key=lambda x: len(x[0]), reverse=True)
    word_ids, word_labels, src_str_word = zip(*word_data)

    char_ids = [torch.LongTensor(e["char_ids"]) for e in data]
    src_str_char = [e["text"] for e in data]
    char_labels = [torch.LongTensor(e["labels"]) for e in data]
    char_data = list(zip(char_ids, char_labels, src_str_char))
    char_data.sort(key=lambda x: len(x[0]), reverse=True)
    char_ids, char_labels, src_str_char = zip(*char_data)

    word_tensor = nn.utils.rnn.pad_sequence(word_ids, batch_first= True, padding_value= pad_id).type(torch.LongTensor)
    char_tensor = nn.utils.rnn.pad_sequence(char_ids, batch_first= True, padding_value= pad_id).type(torch.LongTensor)

    return {"word_ids": word_ids, \
            "char_ids": char_ids, \
            "word_text": src_str_word, \
            "char_text": src_str_char, \
            "word_tensor": word_tensor.to(device), \
            "char_tensor": char_tensor.to(device), \
            "word_labels": torch.stack(word_labels).to(device), \
            "char_labels": torch.stack(char_labels).to(device)}
    #return {"conv_ids":(src_ids, tgt_ids), "conv":(src_str, tgt_str), "conv_tensors":(src_seqs.to(device), tgt_seqs.to(device))}

In [ ]:
max_slen = 128
max_clen = max_slen * 5
batch_size = 5

dataset = ToxicDataset(text, labels, vocab, max_slen, max_clen, device)

print(len(dataset[longest_sentence_index]["word_ids"]))
print(len(dataset[longest_sentence_index]["char_ids"]))
print(dataset[longest_sentence_index]["text"][0:100])

128
640
== school sucks == 

 school sucks!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


# Validation Set and Eval Methods

In [ ]:
print("Length of text validation set", len(text_val))
text_val_word_ids = []
text_val_char_ids = []
labels_val_ids = []

for src in text_val:
  keep_len = min(len(vocab.get_word_ids_from_sentence(src)), max_slen)
  text_val_word_ids.append(torch.LongTensor(vocab.get_word_ids_from_sentence(src)[:keep_len]))

  keep_len = min(len(vocab.get_char_ids_from_sentence(src)), max_clen)
  text_val_char_ids.append(torch.LongTensor(vocab.get_char_ids_from_sentence(src)[:keep_len]))


Length of text validation set 70000


In [ ]:
def eval(model, ids, labels):

  ids = nn.utils.rnn.pad_sequence(ids, batch_first= True, padding_value= pad_id).type(torch.LongTensor).to(device)
  predictions = np.zeros((ids.shape[0], 6))

  split = 1000

  for i in range(split):
    start = i * predictions.shape[0] // split
    end = (i+1) * predictions.shape[0] // split
    predictions[start:end] = model.inference(ids[start:end]).cpu().detach().numpy().astype(int)

  labels = np.array(labels).astype(int)
  print("Micro Accuracy: ", accuracy_score(labels.flatten(), predictions.flatten()))
  print("Subset Accuracy: ", accuracy_score(labels, predictions))
  print(classification_report(labels, predictions, labels=[0, 1, 2, 3, 4, 5], digits = 4, zero_division=0))
  return classification_report(labels, predictions, labels=[0, 1, 2, 3, 4, 5], digits = 4, output_dict=True, zero_division=0)


In [ ]:
predictions = np.zeros((70000, 6))
labels = np.array(labels_val)
print("Micro Accuracy: ", accuracy_score(labels.flatten(), predictions.flatten()))
print("Subset Accuracy: ", accuracy_score(labels, predictions))

print(classification_report(predictions, labels, labels=[0, 1, 2, 3, 4, 5], digits = 4, zero_division=0))

Micro Accuracy:  0.9628309523809524
Subset Accuracy:  0.8989714285714285
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         0
           1     0.0000    0.0000    0.0000         0
           2     0.0000    0.0000    0.0000         0
           3     0.0000    0.0000    0.0000         0
           4     0.0000    0.0000    0.0000         0
           5     0.0000    0.0000    0.0000         0

   micro avg     0.0000    0.0000    0.0000         0
   macro avg     0.0000    0.0000    0.0000         0
weighted avg     0.0000    0.0000    0.0000         0
 samples avg     0.0000    0.0000    0.0000         0



# Neural Bag of Words Baseline

In [ ]:
class NBOW(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=64, NUM_CLASSES=6):
        super(NBOW, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.F = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.W = nn.Linear(DIM_EMB, NUM_CLASSES)
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, X):
      X = self.F(X)
      X = torch.mean(X, dim = -2)
      X = self.W(X)
      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())

      return loss

    def inference(self, X):
      self.eval()

      out = self.sigmoid(self.forward(X))
      half_threshold = 0.5
      pred = torch.where(out > half_threshold, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, model_file, learning_rate = 0.001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 50.0
      self.train()
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              
              total_loss = 0.0
              self.train()
              for i, batch_data in enumerate(batch_iterator, start=1):
                word_tensors = batch_data["word_tensor"]
                word_labels = batch_data["word_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(word_tensors)

                loss = self.compute_loss(output, word_labels)

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_word_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores


In [ ]:
optimizer = torch.optim.Adam
num_epochs = 4
batch_size = 32
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)
neural_bow = NBOW(vocab.num_words).to(device)

nbow_scores = neural_bow.trainer(data_loader, num_epochs, optimizer, None, learning_rate= 0.01)


Micro Accuracy:  0.97505
Subset Accuracy:  0.9063714285714286
              precision    recall  f1-score   support

           0     0.8365    0.5217    0.6426      6718
           1     0.4713    0.0650    0.1142       631
           2     0.8391    0.4733    0.6052      3835
           3     0.2955    0.0586    0.0977       222
           4     0.7685    0.3622    0.4923      3548
           5     0.2000    0.0152    0.0283       657

   micro avg     0.8127    0.4272    0.5600     15611
   macro avg     0.5685    0.2493    0.3301     15611
weighted avg     0.7725    0.4272    0.5443     15611
 samples avg     0.0476    0.0364    0.0390     15611




Micro Accuracy:  0.9764571428571429
Subset Accuracy:  0.9057857142857143
              precision    recall  f1-score   support

           0     0.7790    0.6030    0.6798      6718
           1     0.4978    0.1775    0.2617       631
           2     0.8224    0.5724    0.6750      3835
           3     0.4035    0.1036    0.1649       222
           4     0.7919    0.4504    0.5742      3548
           5     0.2963    0.0731    0.1172       657

   micro avg     0.7770    0.5142    0.6188     15611
   macro avg     0.5985    0.3300    0.4121     15611
weighted avg     0.7556    0.5142    0.6067     15611
 samples avg     0.0546    0.0452    0.0470     15611




Micro Accuracy:  0.9767761904761905
Subset Accuracy:  0.9064285714285715
              precision    recall  f1-score   support

           0     0.7832    0.6121    0.6872      6718
           1     0.4765    0.2409    0.3200       631
           2     0.8107    0.6042    0.6924      3835
           3     0.3793    0.1982    0.2604       222
           4     0.7704    0.4983    0.6052      3548
           5     0.3498    0.1613    0.2208       657

   micro avg     0.7629    0.5444    0.6354     15611
   macro avg     0.5950    0.3858    0.4643     15611
weighted avg     0.7507    0.5444    0.6293     15611
 samples avg     0.0550    0.0477    0.0487     15611




Micro Accuracy:  0.9766595238095238
Subset Accuracy:  0.9052142857142857
              precision    recall  f1-score   support

           0     0.7823    0.6040    0.6817      6718
           1     0.5233    0.1426    0.2242       631
           2     0.8155    0.5924    0.6863      3835
           3     0.4559    0.1396    0.2138       222
           4     0.7705    0.4853    0.5955      3548
           5     0.3500    0.1598    0.2194       657

   micro avg     0.7702    0.5303    0.6281     15611
   macro avg     0.6162    0.3540    0.4368     15611
weighted avg     0.7545    0.5303    0.6187     15611
 samples avg     0.0556    0.0465    0.0482     15611




# CNBOW

In [ ]:
class CNBOW(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=64, NUM_CLASSES=6):
        super(CNBOW, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.F = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.W = nn.Linear(DIM_EMB, NUM_CLASSES)
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, X):
      X = self.F(X)
      X = torch.mean(X, dim = -2)
      X = self.W(X)
      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())

      return loss

    def inference(self, X):
      self.eval()

      out = self.sigmoid(self.forward(X))
      half_threshold = 0.5
      pred = torch.where(out > half_threshold, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, model_file, learning_rate = 0.001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 50.0
      self.train()
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              
              total_loss = 0.0
              self.train()
              for i, batch_data in enumerate(batch_iterator, start=1):
                char_tensors = batch_data["char_tensor"]
                char_labels = batch_data["char_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(char_tensors)

                loss = self.compute_loss(output, char_labels)

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_char_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores


In [ ]:
optimizer = torch.optim.Adam
num_epochs = 4
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)
neural_cbow = CNBOW(vocab.num_char).to(device)

cnbow_scores = neural_cbow.trainer(data_loader, num_epochs, optimizer, None, learning_rate= 0.01)


Micro Accuracy:  0.9635404761904762
Subset Accuracy:  0.8978857142857143
              precision    recall  f1-score   support

           0     0.6835    0.0543    0.1007      6718
           1     0.4625    0.0586    0.1041       631
           2     0.6632    0.0503    0.0936      3835
           3     0.0000    0.0000    0.0000       222
           4     0.5826    0.0397    0.0744      3548
           5     0.2000    0.0091    0.0175       657

   micro avg     0.6256    0.0475    0.0883     15611
   macro avg     0.4320    0.0354    0.0650     15611
weighted avg     0.6166    0.0475    0.0882     15611
 samples avg     0.0047    0.0036    0.0037     15611




Micro Accuracy:  0.9636476190476191
Subset Accuracy:  0.8982571428571429
              precision    recall  f1-score   support

           0     0.7146    0.0496    0.0927      6718
           1     0.5000    0.0634    0.1125       631
           2     0.7245    0.0555    0.1032      3835
           3     0.1304    0.0135    0.0245       222
           4     0.6213    0.0411    0.0772      3548
           5     0.2157    0.0167    0.0311       657

   micro avg     0.6493    0.0478    0.0890     15611
   macro avg     0.4844    0.0400    0.0735     15611
weighted avg     0.6578    0.0478    0.0890     15611
 samples avg     0.0045    0.0035    0.0036     15611




Micro Accuracy:  0.9635928571428571
Subset Accuracy:  0.8978571428571429
              precision    recall  f1-score   support

           0     0.7000    0.0479    0.0897      6718
           1     0.5263    0.0475    0.0872       631
           2     0.7078    0.0568    0.1052      3835
           3     0.0000    0.0000    0.0000       222
           4     0.5964    0.0279    0.0533      3548
           5     0.2500    0.0137    0.0260       657

   micro avg     0.6544    0.0434    0.0815     15611
   macro avg     0.4634    0.0323    0.0602     15611
weighted avg     0.6425    0.0434    0.0812     15611
 samples avg     0.0044    0.0032    0.0034     15611




Micro Accuracy:  0.9636333333333333
Subset Accuracy:  0.8971857142857143
              precision    recall  f1-score   support

           0     0.6435    0.0661    0.1199      6718
           1     0.5224    0.0555    0.1003       631
           2     0.6486    0.0592    0.1085      3835
           3     0.0000    0.0000    0.0000       222
           4     0.5789    0.0434    0.0808      3548
           5     0.4545    0.0076    0.0150       657

   micro avg     0.6210    0.0554    0.1017     15611
   macro avg     0.4747    0.0386    0.0707     15611
weighted avg     0.6081    0.0554    0.1013     15611
 samples avg     0.0060    0.0044    0.0047     15611




# CHAR-CONV

In [ ]:
class CHAR_CONV(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=32, NUM_FILTERS = 2048, KERNEL_SIZE =4,  NUM_CLASSES=6):
        super(CHAR_CONV, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.embeddings = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.dropout = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv = nn.Conv1d(DIM_EMB, NUM_FILTERS, KERNEL_SIZE)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.linear = nn.Linear(NUM_FILTERS, NUM_CLASSES)

        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()

        torch.nn.init.xavier_uniform(self.conv.weight)
        torch.nn.init.xavier_uniform(self.embeddings.weight)
        # torch.nn.init.xavier_uniform(self.linear)


    def forward(self, X):
      X = self.embeddings(X)
      X = self.dropout(X)
      X = X.permute(0, 2, 1)
      X = self.conv(X)
      X = self.dropout2(X)
      X = self.pool(X).squeeze(-1)
      X = self.linear(X)
      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())
      return loss

    def inference(self, X):
      self.eval()

      out = self.sigmoid(self.forward(X))
      half_threshold = 0.5
      pred = torch.where(out > half_threshold, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, model_file, learning_rate = 0.0001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 100
      
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              self.train()
              total_loss = 0.0
              for i, batch_data in enumerate(batch_iterator, start=1):
                char_tensors = batch_data["char_tensor"]
                char_labels = batch_data["char_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(char_tensors)

                loss = self.compute_loss(output, char_labels.to(device))

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_char_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores

In [ ]:
optimizer = torch.optim.Adam
num_epochs = 6
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

CHAR_CONV = CHAR_CONV(vocab.num_char).to(device)
CHAR_CONV_scores = CHAR_CONV.trainer(data_loader, num_epochs, optimizer, None, learning_rate= 0.0005)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.



Micro Accuracy:  0.973202380952381
Subset Accuracy:  0.8944714285714286
              precision    recall  f1-score   support

           0     0.6930    0.6299    0.6600      6718
           1     0.3307    0.4564    0.3835       631
           2     0.7238    0.7046    0.7141      3835
           3     0.1486    0.1667    0.1571       222
           4     0.6427    0.6195    0.6309      3548
           5     0.3622    0.3562    0.3592       657

   micro avg     0.6449    0.6208    0.6326     15611
   macro avg     0.4835    0.4889    0.4841     15611
weighted avg     0.6528    0.6208    0.6357     15611
 samples avg     0.0512    0.0534    0.0496     15611




Micro Accuracy:  0.97045
Subset Accuracy:  0.8797
              precision    recall  f1-score   support

           0     0.6194    0.7462    0.6769      6718
           1     0.2682    0.7116    0.3896       631
           2     0.6246    0.8235    0.7104      3835
           3     0.2783    0.2883    0.2832       222
           4     0.6298    0.7063    0.6659      3548
           5     0.4026    0.4658    0.4319       657

   micro avg     0.5808    0.7364    0.6494     15611
   macro avg     0.4705    0.6236    0.5263     15611
weighted avg     0.5949    0.7364    0.6551     15611
 samples avg     0.0580    0.0664    0.0590     15611




Micro Accuracy:  0.971297619047619
Subset Accuracy:  0.8943857142857143
              precision    recall  f1-score   support

           0     0.7778    0.6399    0.7022      6718
           1     0.2571    0.7940    0.3884       631
           2     0.6394    0.8318    0.7230      3835
           3     0.1490    0.6532    0.2427       222
           4     0.6154    0.7477    0.6751      3548
           5     0.3641    0.5525    0.4389       657

   micro avg     0.5948    0.7143    0.6491     15611
   macro avg     0.4671    0.7032    0.5284     15611
weighted avg     0.6595    0.7143    0.6709     15611
 samples avg     0.0474    0.0607    0.0509     15611




Micro Accuracy:  0.9670404761904762
Subset Accuracy:  0.8807142857142857
              precision    recall  f1-score   support

           0     0.6582    0.7583    0.7047      6718
           1     0.2462    0.8146    0.3781       631
           2     0.5937    0.8733    0.7068      3835
           3     0.1767    0.6486    0.2777       222
           4     0.5895    0.7833    0.6727      3548
           5     0.2468    0.7382    0.3699       657

   micro avg     0.5385    0.7921    0.6411     15611
   macro avg     0.4185    0.7694    0.5183     15611
weighted avg     0.5859    0.7921    0.6646     15611
 samples avg     0.0541    0.0712    0.0587     15611




Micro Accuracy:  0.9732119047619048
Subset Accuracy:  0.8906285714285714
              precision    recall  f1-score   support

           0     0.6602    0.7657    0.7091      6718
           1     0.3374    0.5246    0.4107       631
           2     0.6532    0.8417    0.7356      3835
           3     0.3441    0.3829    0.3625       222
           4     0.6367    0.7379    0.6836      3548
           5     0.3483    0.6621    0.4565       657

   micro avg     0.6128    0.7585    0.6779     15611
   macro avg     0.4967    0.6525    0.5596     15611
weighted avg     0.6225    0.7585    0.6822     15611
 samples avg     0.0609    0.0688    0.0617     15611




Micro Accuracy:  0.972302380952381
Subset Accuracy:  0.8943142857142857
              precision    recall  f1-score   support

           0     0.7265    0.7236    0.7250      6718
           1     0.3478    0.5087    0.4131       631
           2     0.6223    0.8634    0.7232      3835
           3     0.3460    0.4910    0.4060       222
           4     0.5998    0.7773    0.6771      3548
           5     0.2607    0.7412    0.3857       657

   micro avg     0.6009    0.7589    0.6707     15611
   macro avg     0.4838    0.6842    0.5550     15611
weighted avg     0.6318    0.7589    0.6823     15611
 samples avg     0.0550    0.0674    0.0580     15611




# WORD-CONV

In [ ]:
class WORD_CONV(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=96, NUM_FILTERS = 2048, KERNEL_SIZE =2,  NUM_CLASSES=6):
        super(WORD_CONV, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.embeddings = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.dropout = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv = nn.Conv1d(DIM_EMB, NUM_FILTERS, KERNEL_SIZE)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.linear = nn.Linear(NUM_FILTERS, NUM_CLASSES)
        
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()

        torch.nn.init.xavier_uniform(self.conv.weight)
        torch.nn.init.xavier_uniform(self.embeddings.weight)
        # torch.nn.init.xavier_uniform(self.linear)


    def forward(self, X):
      X = self.embeddings(X)
      X = self.dropout(X)
      X = X.permute(0, 2, 1)
      X = self.conv(X)
      X = self.dropout2(X)
      X = self.pool(X).squeeze(-1)
      X = self.linear(X)
      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())
      return loss

    def inference(self, X):
      self.eval()

      out = self.sigmoid(self.forward(X))
      half_threshold = 0.5
      pred = torch.where(out > half_threshold, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, model_file, learning_rate = 0.0001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 100
      
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              self.train()
              total_loss = 0.0
              for i, batch_data in enumerate(batch_iterator, start=1):
                word_tensors = batch_data["word_tensor"]
                word_labels = batch_data["word_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(word_tensors)

                loss = self.compute_loss(output, word_labels.to(device))

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_word_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores

In [ ]:
optimizer = torch.optim.Adam
num_epochs = 6
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

WORD_CONV = WORD_CONV(vocab.num_words).to(device)
WORD_CONV_scores = WORD_CONV.trainer(data_loader, num_epochs, optimizer, None, learning_rate= 0.0005)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.



Micro Accuracy:  0.9670285714285715
Subset Accuracy:  0.8776428571428572
              precision    recall  f1-score   support

           0     0.6328    0.8284    0.7175      6718
           1     0.2715    0.6751    0.3873       631
           2     0.5872    0.8913    0.7080      3835
           3     0.0526    0.0180    0.0268       222
           4     0.5196    0.8382    0.6415      3548
           5     0.1894    0.4764    0.2710       657

   micro avg     0.5373    0.8135    0.6472     15611
   macro avg     0.3755    0.6212    0.4587     15611
weighted avg     0.5543    0.8135    0.6559     15611
 samples avg     0.0600    0.0762    0.0638     15611




Micro Accuracy:  0.9665547619047619
Subset Accuracy:  0.8647142857142858
              precision    recall  f1-score   support

           0     0.5595    0.8514    0.6752      6718
           1     0.3036    0.6212    0.4079       631
           2     0.6561    0.8506    0.7408      3835
           3     0.2199    0.6081    0.3230       222
           4     0.5130    0.8424    0.6377      3548
           5     0.3106    0.6301    0.4161       657

   micro avg     0.5322    0.8271    0.6477     15611
   macro avg     0.4271    0.7340    0.5334     15611
weighted avg     0.5470    0.8271    0.6561     15611
 samples avg     0.0646    0.0787    0.0677     15611




Micro Accuracy:  0.9666095238095238
Subset Accuracy:  0.8587571428571429
              precision    recall  f1-score   support

           0     0.5269    0.8288    0.6443      6718
           1     0.3102    0.5800    0.4042       631
           2     0.6377    0.8605    0.7325      3835
           3     0.2665    0.6712    0.3816       222
           4     0.5818    0.7506    0.6555      3548
           5     0.3321    0.6728    0.4447       657

   micro avg     0.5339    0.7999    0.6404     15611
   macro avg     0.4425    0.7273    0.5438     15611
weighted avg     0.5459    0.7999    0.6467     15611
 samples avg     0.0652    0.0758    0.0668     15611




Micro Accuracy:  0.9498857142857143
Subset Accuracy:  0.7818428571428572
              precision    recall  f1-score   support

           0     0.3690    0.8906    0.5218      6718
           1     0.2419    0.7607    0.3671       631
           2     0.6384    0.8488    0.7287      3835
           3     0.2987    0.6216    0.4035       222
           4     0.5200    0.8047    0.6318      3548
           5     0.1971    0.7960    0.3159       657

   micro avg     0.4148    0.8477    0.5570     15611
   macro avg     0.3775    0.7871    0.4948     15611
weighted avg     0.4561    0.8477    0.5810     15611
 samples avg     0.0657    0.0824    0.0695     15611




Micro Accuracy:  0.9579595238095238
Subset Accuracy:  0.8338142857142857
              precision    recall  f1-score   support

           0     0.4591    0.7638    0.5735      6718
           1     0.2857    0.6244    0.3920       631
           2     0.5948    0.8274    0.6920      3835
           3     0.3620    0.5495    0.4365       222
           4     0.4522    0.8044    0.5789      3548
           5     0.2512    0.6575    0.3635       657

   micro avg     0.4610    0.7755    0.5783     15611
   macro avg     0.4008    0.7045    0.5061     15611
weighted avg     0.4737    0.7755    0.5857     15611
 samples avg     0.0583    0.0729    0.0616     15611




Micro Accuracy:  0.9407714285714286
Subset Accuracy:  0.7583428571428571
              precision    recall  f1-score   support

           0     0.3243    0.8150    0.4639      6718
           1     0.2810    0.6133    0.3855       631
           2     0.5437    0.8297    0.6570      3835
           3     0.1439    0.7117    0.2394       222
           4     0.4244    0.7785    0.5493      3548
           5     0.1898    0.6865    0.2974       657

   micro avg     0.3641    0.7953    0.4995     15611
   macro avg     0.3179    0.7391    0.4321     15611
weighted avg     0.3910    0.7953    0.5174     15611
 samples avg     0.0602    0.0766    0.0638     15611




# WORD GRU

In [ ]:
class WORD_GRU(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=128, GRU_HID_DIM = 256, GRU_LAYERS = 2,  NUM_CLASSES=6):
        super(WORD_GRU, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.embeddings = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.GRU = nn.GRU(DIM_EMB, GRU_HID_DIM, GRU_LAYERS, batch_first = True, dropout = 0.5, bidirectional = True)
        self.linear1 = nn.Linear( GRU_HID_DIM * 2, NUM_CLASSES)

        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()



    def forward(self, X):
      X = self.embeddings(X)
      X, _ = self.GRU(X)
      X = self.linear1(X[:, -1])
      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())
      return loss

    def inference(self, X):
      self.eval()

      out = self.sigmoid(self.forward(X))
      half_threshold = 0.5
      pred = torch.where(out > half_threshold, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, model_file, learning_rate = 0.0001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 100
      
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              
              total_loss = 0.0
              self.train()
              for i, batch_data in enumerate(batch_iterator, start=1):
                word_tensors = batch_data["word_tensor"]
                word_labels = batch_data["word_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(word_tensors)

                loss = self.compute_loss(output, word_labels.to(device))

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_word_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores

In [ ]:
optimizer = torch.optim.Adam
num_epochs = 10
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

WORD_GRU = WORD_GRU(vocab.num_words).to(device)
WORD_GRU_scores = WORD_GRU.trainer(data_loader, num_epochs, optimizer, None, learning_rate= 0.0005)


Micro Accuracy:  0.9785357142857143
Subset Accuracy:  0.9120285714285714
              precision    recall  f1-score   support

           0     0.8387    0.6091    0.7057      6718
           1     0.4896    0.3344    0.3974       631
           2     0.8695    0.6047    0.7133      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7929    0.5104    0.6211      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.8211    0.5402    0.6517     15611
   macro avg     0.4984    0.3431    0.4062     15611
weighted avg     0.7745    0.5402    0.6361     15611
 samples avg     0.0546    0.0464    0.0478     15611




Micro Accuracy:  0.9802547619047619
Subset Accuracy:  0.9156857142857143
              precision    recall  f1-score   support

           0     0.8361    0.6646    0.7406      6718
           1     0.4626    0.4707    0.4666       631
           2     0.8521    0.6834    0.7585      3835
           3     0.2500    0.0045    0.0088       222
           4     0.8013    0.5854    0.6765      3548
           5     0.6786    0.1446    0.2384       657

   micro avg     0.8102    0.6121    0.6974     15611
   macro avg     0.6468    0.4255    0.4816     15611
weighted avg     0.8021    0.6121    0.6878     15611
 samples avg     0.0589    0.0531    0.0535     15611




Micro Accuracy:  0.9807261904761905
Subset Accuracy:  0.9185285714285715
              precision    recall  f1-score   support

           0     0.8703    0.6182    0.7229      6718
           1     0.5802    0.1949    0.2918       631
           2     0.8322    0.7205    0.7723      3835
           3     0.7407    0.1802    0.2899       222
           4     0.7973    0.6153    0.6946      3548
           5     0.6325    0.3196    0.4247       657

   micro avg     0.8288    0.6068    0.7006     15611
   macro avg     0.7422    0.4414    0.5327     15611
weighted avg     0.8208    0.6068    0.6925     15611
 samples avg     0.0549    0.0512    0.0510     15611




Micro Accuracy:  0.9812333333333333
Subset Accuracy:  0.9183428571428571
              precision    recall  f1-score   support

           0     0.8284    0.6890    0.7523      6718
           1     0.5989    0.1727    0.2681       631
           2     0.8455    0.7064    0.7697      3835
           3     0.7195    0.2658    0.3882       222
           4     0.7610    0.6801    0.7183      3548
           5     0.7458    0.2725    0.3991       657

   micro avg     0.8100    0.6469    0.7193     15611
   macro avg     0.7498    0.4644    0.5493     15611
weighted avg     0.8030    0.6469    0.7092     15611
 samples avg     0.0613    0.0573    0.0571     15611




Micro Accuracy:  0.9804880952380952
Subset Accuracy:  0.9127
              precision    recall  f1-score   support

           0     0.7630    0.7542    0.7586      6718
           1     0.5039    0.3059    0.3807       631
           2     0.8479    0.6965    0.7648      3835
           3     0.6481    0.3153    0.4242       222
           4     0.7626    0.6483    0.7008      3548
           5     0.6647    0.3440    0.4534       657

   micro avg     0.7719    0.6743    0.7198     15611
   macro avg     0.6984    0.5107    0.5804     15611
weighted avg     0.7675    0.6743    0.7141     15611
 samples avg     0.0672    0.0619    0.0619     15611




Micro Accuracy:  0.9802880952380952
Subset Accuracy:  0.9140714285714285
              precision    recall  f1-score   support

           0     0.8057    0.6931    0.7451      6718
           1     0.4467    0.4453    0.4460       631
           2     0.8496    0.6866    0.7594      3835
           3     0.6863    0.3153    0.4321       222
           4     0.7646    0.6353    0.6940      3548
           5     0.6399    0.3516    0.4538       657

   micro avg     0.7838    0.6486    0.7098     15611
   macro avg     0.6988    0.5212    0.5884     15611
weighted avg     0.7840    0.6486    0.7082     15611
 samples avg     0.0609    0.0574    0.0567     15611




Micro Accuracy:  0.98015
Subset Accuracy:  0.9125714285714286
              precision    recall  f1-score   support

           0     0.7767    0.7234    0.7491      6718
           1     0.4750    0.3772    0.4205       631
           2     0.8332    0.6996    0.7606      3835
           3     0.5389    0.4054    0.4627       222
           4     0.7796    0.6170    0.6888      3548
           5     0.6482    0.3562    0.4597       657

   micro avg     0.7732    0.6594    0.7118     15611
   macro avg     0.6753    0.5298    0.5902     15611
weighted avg     0.7703    0.6594    0.7087     15611
 samples avg     0.0640    0.0594    0.0591     15611




Micro Accuracy:  0.9795619047619047
Subset Accuracy:  0.9102857142857143
              precision    recall  f1-score   support

           0     0.7515    0.7404    0.7459      6718
           1     0.5000    0.2108    0.2965       631
           2     0.8240    0.7080    0.7616      3835
           3     0.6146    0.2658    0.3711       222
           4     0.7302    0.6705    0.6991      3548
           5     0.5368    0.4551    0.4926       657

   micro avg     0.7493    0.6764    0.7110     15611
   macro avg     0.6595    0.5084    0.5611     15611
weighted avg     0.7433    0.6764    0.7050     15611
 samples avg     0.0652    0.0616    0.0607     15611




Micro Accuracy:  0.978895238095238
Subset Accuracy:  0.9059571428571429
              precision    recall  f1-score   support

           0     0.7112    0.7764    0.7424      6718
           1     0.4728    0.3027    0.3691       631
           2     0.8125    0.7265    0.7671      3835
           3     0.6290    0.3514    0.4509       222
           4     0.7078    0.6922    0.6999      3548
           5     0.5563    0.4064    0.4697       657

   micro avg     0.7213    0.7042    0.7127     15611
   macro avg     0.6483    0.5426    0.5832     15611
weighted avg     0.7180    0.7042    0.7081     15611
 samples avg     0.0679    0.0655    0.0640     15611




Micro Accuracy:  0.97955
Subset Accuracy:  0.9114142857142857
              precision    recall  f1-score   support

           0     0.7729    0.7091    0.7396      6718
           1     0.5174    0.2361    0.3243       631
           2     0.8001    0.7359    0.7666      3835
           3     0.6531    0.2883    0.4000       222
           4     0.7079    0.6781    0.6927      3548
           5     0.6257    0.3333    0.4350       657

   micro avg     0.7539    0.6677    0.7082     15611
   macro avg     0.6795    0.4968    0.5597     15611
weighted avg     0.7466    0.6677    0.7012     15611
 samples avg     0.0626    0.0601    0.0588     15611




# CHAR GRU


In [ ]:
class CHAR_GRU(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=32, GRU_HID_DIM = 128, GRU_LAYERS = 2,  NUM_CLASSES=6):
        super(CHAR_GRU, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.embeddings = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.GRU = nn.GRU(DIM_EMB, GRU_HID_DIM, GRU_LAYERS, batch_first = True, dropout = 0.5, bidirectional = True)
        self.linear1 = nn.Linear( GRU_HID_DIM * 2, NUM_CLASSES)

        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()


    def forward(self, X):
      X = self.embeddings(X)
      X, _ = self.GRU(X)
      X = self.linear1(X[:, -1])
      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())
      return loss

    def inference(self, X):
      self.eval()

      out = self.sigmoid(self.forward(X))
      half_threshold = 0.5
      pred = torch.where(out > half_threshold, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, model_file, learning_rate = 0.0001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 100
      
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              
              total_loss = 0.0
              self.train()
              for i, batch_data in enumerate(batch_iterator, start=1):
                char_tensors = batch_data["char_tensor"]
                char_labels = batch_data["char_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(char_tensors)

                loss = self.compute_loss(output, char_labels.to(device))

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_char_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores

In [ ]:
optimizer = torch.optim.Adam
num_epochs = 10
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

CHAR_GRU = CHAR_GRU(vocab.num_char).to(device)
CHAR_GRU_scores = CHAR_GRU.trainer(data_loader, num_epochs, optimizer, None, learning_rate= 0.0005)


Micro Accuracy:  0.9726
Subset Accuracy:  0.9036714285714286
              precision    recall  f1-score   support

           0     0.7997    0.3446    0.4816      6718
           1     0.7778    0.0111    0.0219       631
           2     0.8418    0.4884    0.6182      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7357    0.3703    0.4927      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7967    0.3529    0.4891     15611
   macro avg     0.5258    0.2024    0.2691     15611
weighted avg     0.7496    0.3529    0.4720     15611
 samples avg     0.0306    0.0274    0.0278     15611




Micro Accuracy:  0.9782142857142857
Subset Accuracy:  0.9124
              precision    recall  f1-score   support

           0     0.8325    0.5795    0.6833      6718
           1     0.5321    0.1838    0.2733       631
           2     0.8070    0.6934    0.7459      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7148    0.6161    0.6618      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7872    0.5672    0.6593     15611
   macro avg     0.4811    0.3455    0.3940     15611
weighted avg     0.7405    0.5672    0.6388     15611
 samples avg     0.0503    0.0476    0.0469     15611




Micro Accuracy:  0.9790047619047619
Subset Accuracy:  0.9104571428571429
              precision    recall  f1-score   support

           0     0.8047    0.6416    0.7139      6718
           1     0.5449    0.1442    0.2281       631
           2     0.8624    0.6472    0.7395      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7680    0.5767    0.6587      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.8070    0.5720    0.6694     15611
   macro avg     0.4967    0.3349    0.3900     15611
weighted avg     0.7547    0.5720    0.6478     15611
 samples avg     0.0576    0.0494    0.0509     15611




Micro Accuracy:  0.9798476190476191
Subset Accuracy:  0.9157428571428572
              precision    recall  f1-score   support

           0     0.8685    0.5907    0.7031      6718
           1     0.5415    0.1965    0.2884       631
           2     0.8641    0.6681    0.7535      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7932    0.5784    0.6689      3548
           5     0.7053    0.2222    0.3380       657

   micro avg     0.8385    0.5670    0.6766     15611
   macro avg     0.6288    0.3760    0.4587     15611
weighted avg     0.8178    0.5670    0.6656     15611
 samples avg     0.0529    0.0470    0.0478     15611




Micro Accuracy:  0.9801761904761904
Subset Accuracy:  0.9166714285714286
              precision    recall  f1-score   support

           0     0.8892    0.5831    0.7043      6718
           1     0.5690    0.2155    0.3126       631
           2     0.8471    0.7108    0.7730      3835
           3     0.0000    0.0000    0.0000       222
           4     0.8307    0.5296    0.6468      3548
           5     0.6240    0.3562    0.4535       657

   micro avg     0.8469    0.5696    0.6811     15611
   macro avg     0.6267    0.3992    0.4817     15611
weighted avg     0.8288    0.5696    0.6717     15611
 samples avg     0.0523    0.0471    0.0477     15611




Micro Accuracy:  0.9811
Subset Accuracy:  0.9185142857142857
              precision    recall  f1-score   support

           0     0.8490    0.6581    0.7415      6718
           1     0.6347    0.1680    0.2657       631
           2     0.8351    0.7327    0.7806      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7577    0.6697    0.7110      3548
           5     0.6713    0.3638    0.4719       657

   micro avg     0.8137    0.6375    0.7149     15611
   macro avg     0.6246    0.4320    0.4951     15611
weighted avg     0.7966    0.6375    0.7030     15611
 samples avg     0.0578    0.0549    0.0542     15611




Micro Accuracy:  0.9810761904761904
Subset Accuracy:  0.9176714285714286
              precision    recall  f1-score   support

           0     0.8344    0.6736    0.7454      6718
           1     0.7363    0.1062    0.1856       631
           2     0.8209    0.7447    0.7810      3835
           3     0.7600    0.0856    0.1538       222
           4     0.7723    0.6471    0.7042      3548
           5     0.7270    0.3120    0.4366       657

   micro avg     0.8122    0.6385    0.7150     15611
   macro avg     0.7751    0.4282    0.5011     15611
weighted avg     0.8074    0.6385    0.7007     15611
 samples avg     0.0599    0.0559    0.0557     15611




Micro Accuracy:  0.9812928571428572
Subset Accuracy:  0.9187
              precision    recall  f1-score   support

           0     0.8447    0.6743    0.7499      6718
           1     0.5040    0.2964    0.3733       631
           2     0.8240    0.7507    0.7856      3835
           3     0.8571    0.0811    0.1481       222
           4     0.7747    0.6530    0.7087      3548
           5     0.6618    0.4110    0.5070       657

   micro avg     0.8065    0.6534    0.7220     15611
   macro avg     0.7444    0.4777    0.5454     15611
weighted avg     0.8024    0.6534    0.7153     15611
 samples avg     0.0594    0.0567    0.0558     15611




Micro Accuracy:  0.9815
Subset Accuracy:  0.9182571428571429
              precision    recall  f1-score   support

           0     0.8290    0.6972    0.7574      6718
           1     0.5574    0.2694    0.3632       631
           2     0.8630    0.6999    0.7729      3835
           3     0.5897    0.2072    0.3067       222
           4     0.7856    0.6505    0.7117      3548
           5     0.6462    0.4642    0.5403       657

   micro avg     0.8123    0.6532    0.7241     15611
   macro avg     0.7118    0.4981    0.5754     15611
weighted avg     0.8054    0.6532    0.7194     15611
 samples avg     0.0620    0.0572    0.0572     15611




Micro Accuracy:  0.9812
Subset Accuracy:  0.9150857142857143
              precision    recall  f1-score   support

           0     0.7808    0.7523    0.7663      6718
           1     0.4864    0.3677    0.4188       631
           2     0.8423    0.7296    0.7819      3835
           3     0.5741    0.2793    0.3758       222
           4     0.7487    0.7111    0.7294      3548
           5     0.6427    0.4353    0.5191       657

   micro avg     0.7718    0.7017    0.7351     15611
   macro avg     0.6791    0.5459    0.5985     15611
weighted avg     0.7679    0.7017    0.7317     15611
 samples avg     0.0659    0.0634    0.0622     15611




# WORD CONV LSTM

In [ ]:
class WORD_OPT(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=128, NUM_FILTERS = 2024, KERNEL_SIZE =2, LSTM_HID_DIM = 256, LSTM_LAYERS = 2,  NUM_CLASSES=6):
      # 128 64 2 originally
        super(WORD_OPT, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.embeddings = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.conv = nn.Conv1d(DIM_EMB, NUM_FILTERS, KERNEL_SIZE)

        self.dropout1 = nn.Dropout(0.2)
        self.LSTM = nn.LSTM(NUM_FILTERS, LSTM_HID_DIM, LSTM_LAYERS, batch_first = True, dropout = 0.5, bidirectional = True)

        self.maxpool = nn.AdaptiveMaxPool1d(1)
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.dropout2 = nn.Dropout(0.2)
        self.linear1 = nn.Linear(LSTM_HID_DIM * 4, 128)
        self.nl1 = nn.ReLU()

        self.dropout3 = nn.Dropout(0.2)
        self.linear2 = nn.Linear(128, NUM_CLASSES)

        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()



    def forward(self, X):
      X = self.embeddings(X)
      X = self.dropout1(X)

      X = X.permute(0, 2, 1)
      X = self.conv(X)
      X = X.permute(0, 2, 1)


      X, _ = self.LSTM(X)
      X = X.permute(0, 2, 1)
      max = self.maxpool(X).squeeze(-1)
      avg = self.avgpool(X).squeeze(-1)
      X = torch.cat((max, avg), dim= -1)

      X = self.dropout2(X)
      X = self.linear1(X)
      X = self.nl1(X)

      X = self.dropout3(X)
      X = self.linear2(X)
      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())
      return loss

    def inference(self, X):
      self.eval()
      pad = torch.zeros(X.shape[0], 5).long().to(device)
      X = torch.cat((X, pad), dim = -1).to(device)
      out = self.sigmoid(self.forward(X)).squeeze(0)
      pred = torch.where(out > 0.5, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, learning_rate = 0.0001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 100
      
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              
              total_loss = 0.0
              self.train()
              for i, batch_data in enumerate(batch_iterator, start=1):
                word_tensors = batch_data["word_tensor"]
                word_labels = batch_data["word_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(word_tensors)

                loss = self.compute_loss(output, word_labels.to(device))

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_word_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores

In [ ]:
optimizer = torch.optim.Adam
num_epochs = 15
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

WORD_opt = WORD_OPT(vocab.num_words).to(device)
WORD_opt_scores = WORD_opt.trainer(data_loader, num_epochs, optimizer, learning_rate= 0.001)


Micro Accuracy:  0.9783357142857143
Subset Accuracy:  0.9123428571428571
              precision    recall  f1-score   support

           0     0.8330    0.6090    0.7036      6718
           1     0.5185    0.0222    0.0426       631
           2     0.7825    0.7119    0.7455      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7025    0.6203    0.6589      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7817    0.5788    0.6651     15611
   macro avg     0.4728    0.3272    0.3584     15611
weighted avg     0.7313    0.5788    0.6374     15611
 samples avg     0.0526    0.0496    0.0487     15611




Micro Accuracy:  0.9793666666666667
Subset Accuracy:  0.9139857142857143
              precision    recall  f1-score   support

           0     0.8441    0.6325    0.7231      6718
           1     0.5307    0.2330    0.3238       631
           2     0.8181    0.7035    0.7565      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7344    0.6032    0.6623      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.8014    0.5915    0.6806     15611
   macro avg     0.4879    0.3620    0.4110     15611
weighted avg     0.7526    0.5915    0.6606     15611
 samples avg     0.0551    0.0508    0.0504     15611




Micro Accuracy:  0.9793071428571428
Subset Accuracy:  0.9120428571428572
              precision    recall  f1-score   support

           0     0.8111    0.6978    0.7502      6718
           1     0.4341    0.4279    0.4310       631
           2     0.7646    0.7739    0.7692      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6984    0.6708    0.6843      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7527    0.6602    0.7034     15611
   macro avg     0.4513    0.4284    0.4391     15611
weighted avg     0.7131    0.6602    0.6847     15611
 samples avg     0.0591    0.0584    0.0561     15611




Micro Accuracy:  0.9794880952380952
Subset Accuracy:  0.9138857142857143
              precision    recall  f1-score   support

           0     0.8085    0.7015    0.7513      6718
           1     0.6374    0.1727    0.2718       631
           2     0.7468    0.7883    0.7670      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6784    0.7255    0.7012      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7527    0.6674    0.7075     15611
   macro avg     0.4785    0.3980    0.4152     15611
weighted avg     0.7114    0.6674    0.6821     15611
 samples avg     0.0596    0.0595    0.0570     15611




Micro Accuracy:  0.9796666666666667
Subset Accuracy:  0.9135857142857143
              precision    recall  f1-score   support

           0     0.8032    0.7176    0.7580      6718
           1     0.5116    0.2092    0.2970       631
           2     0.7902    0.7554    0.7724      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6701    0.7201    0.6942      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7573    0.6665    0.7090     15611
   macro avg     0.4625    0.4004    0.4203     15611
weighted avg     0.7128    0.6665    0.6857     15611
 samples avg     0.0617    0.0599    0.0581     15611



...


Micro Accuracy:  0.9802857142857143
Subset Accuracy:  0.9158714285714286
              precision    recall  f1-score   support

           0     0.8182    0.7050    0.7574      6718
           1     0.4888    0.3471    0.4059       631
           2     0.7747    0.7854    0.7800      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7143    0.7145    0.7144      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7681    0.6727    0.7173     15611
   macro avg     0.4660    0.4253    0.4430     15611
weighted avg     0.7245    0.6727    0.6963     15611
 samples avg     0.0606    0.0599    0.0579     15611




Micro Accuracy:  0.9802642857142857
Subset Accuracy:  0.9146857142857143
              precision    recall  f1-score   support

           0     0.7991    0.7334    0.7648      6718
           1     0.7200    0.0285    0.0549       631
           2     0.7745    0.7721    0.7733      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7275    0.6869    0.7066      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7739    0.6625    0.7139     15611
   macro avg     0.5035    0.3701    0.3833     15611
weighted avg     0.7286    0.6625    0.6819     15611
 samples avg     0.0643    0.0604    0.0597     15611




Micro Accuracy:  0.9804285714285714
Subset Accuracy:  0.9172
              precision    recall  f1-score   support

           0     0.8511    0.6643    0.7462      6718
           1     0.5357    0.2139    0.3058       631
           2     0.7885    0.7682    0.7782      3835
           3     0.0000    0.0000    0.0000       222
           4     0.7546    0.6406    0.6930      3548
           5     1.0000    0.0015    0.0030       657

   micro avg     0.8018    0.6289    0.7049     15611
   macro avg     0.6550    0.3814    0.4210     15611
weighted avg     0.7952    0.6289    0.6823     15611
 samples avg     0.0581    0.0551    0.0544     15611




Micro Accuracy:  0.9804833333333334
Subset Accuracy:  0.9159
              precision    recall  f1-score   support

           0     0.8214    0.7038    0.7581      6718
           1     0.5567    0.1712    0.2618       631
           2     0.7955    0.7437    0.7687      3835
           3     0.6667    0.0631    0.1152       222
           4     0.7843    0.6065    0.6840      3548
           5     0.8000    0.0122    0.0240       657

   micro avg     0.8011    0.6317    0.7064     15611
   macro avg     0.7374    0.3834    0.4353     15611
weighted avg     0.7928    0.6317    0.6838     15611
 samples avg     0.0626    0.0567    0.0571     15611




Micro Accuracy:  0.9802285714285714
Subset Accuracy:  0.9169714285714285
              precision    recall  f1-score   support

           0     0.8676    0.6221    0.7246      6718
           1     0.5385    0.2219    0.3143       631
           2     0.8336    0.7106    0.7672      3835
           3     0.6471    0.0495    0.0921       222
           4     0.7684    0.6229    0.6880      3548
           5     0.9444    0.0259    0.0504       657

   micro avg     0.8246    0.5946    0.6909     15611
   macro avg     0.7666    0.3755    0.4394     15611
weighted avg     0.8235    0.5946    0.6728     15611
 samples avg     0.0548    0.0508    0.0506     15611




# CHAR-CONV-LSTM

In [ ]:
class CHAR_CONV_LSTM(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=32, NUM_FILTERS = 2024, KERNEL_SIZE =4, LSTM_HID_DIM = 256, LSTM_LAYERS = 2,  NUM_CLASSES=6):
        super(CHAR_CONV_LSTM, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        self.VOCAB_SIZE = VOCAB_SIZE
        self.NUM_CLASSES = NUM_CLASSES

        self.embeddings = nn.Embedding(VOCAB_SIZE, DIM_EMB, padding_idx= pad_id)
        self.dropout = nn.Dropout(0.25)
        
        self.conv = nn.Conv1d(DIM_EMB, NUM_FILTERS, KERNEL_SIZE)

        self.LSTM = nn.LSTM(NUM_FILTERS, LSTM_HID_DIM, LSTM_LAYERS, batch_first = True, dropout = 0.5, bidirectional = True)
        self.maxpool = nn.AdaptiveMaxPool1d(1)
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.dropout2 = nn.Dropout(0.5)
        self.linear1 = nn.Linear(LSTM_HID_DIM * 4, 128)
        self.nl1 = nn.ReLU()

        self.dropout3 = nn.Dropout(0.5)
        self.linear2 = nn.Linear(128, NUM_CLASSES)

        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCEWithLogitsLoss()

        torch.nn.init.xavier_uniform(self.conv.weight)
        torch.nn.init.xavier_uniform(self.embeddings.weight)
        # torch.nn.init.xavier_uniform(self.linear)


    def forward(self, X):
      X = self.embeddings(X)
      X = self.dropout(X)
      X = X.permute(0, 2, 1)
      X = self.conv(X)
      X = X.permute(0, 2, 1)
      X, _ = self.LSTM(X)
      X = X.permute(0, 2, 1)
      max = self.maxpool(X).squeeze(-1)
      avg = self.avgpool(X).squeeze(-1)
      X = torch.cat((max, avg), dim= -1)

      X = self.dropout2(X)
      X = self.linear1(X)
      X = self.nl1(X)

      X = self.dropout3(X)
      X = self.linear2(X)


      return X

    def compute_loss(self, output, labels):
      loss = self.loss(output.float(), labels.float())
      return loss

    def inference(self, X):
      self.eval()

      out = self.sigmoid(self.forward(X))
      half_threshold = 0.5
      pred = torch.where(out > half_threshold, 1, 0)
      
      return pred

    def trainer(self, data_loader, num_epochs, optimizer, model_file, learning_rate = 0.0001):
      optimizer = optimizer(self.parameters(), learning_rate)
      clip = 100
      
      scores = []
      for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
          with tqdm.notebook.tqdm(
                  data_loader,
                  desc="epoch {}".format(epoch + 1),
                  unit="batch",
                  total=len(data_loader)) as batch_iterator:
              self.train()
              total_loss = 0.0
              for i, batch_data in enumerate(batch_iterator, start=1):
                char_tensors = batch_data["char_tensor"]
                char_labels = batch_data["char_labels"]

                optimizer.zero_grad()
                self.zero_grad()

                output = self.forward(char_tensors)

                loss = self.compute_loss(output, char_labels.to(device))

                total_loss += loss.item()
                loss.backward()
                # _ = nn.utils.clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

              scores.append(eval(self, text_val_char_ids, labels_val))
      # torch.save(model.state_dict(), model_file)
      return scores

In [ ]:
optimizer = torch.optim.Adam
num_epochs = 30
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

CHAR_CONV_LSTM = CHAR_CONV_LSTM(vocab.num_char).to(device)
CHAR_CONV_LSTM_scores = CHAR_CONV_LSTM.trainer(data_loader, num_epochs, optimizer, None, learning_rate= 0.001)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.



Micro Accuracy:  0.9745
Subset Accuracy:  0.9025571428571428
              precision    recall  f1-score   support

           0     0.7389    0.5674    0.6419      6718
           1     0.0000    0.0000    0.0000       631
           2     0.6949    0.7134    0.7041      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6264    0.6294    0.6279      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.6935    0.5625    0.6212     15611
   macro avg     0.3434    0.3184    0.3290     15611
weighted avg     0.6311    0.5625    0.5919     15611
 samples avg     0.0475    0.0482    0.0457     15611




Micro Accuracy:  0.97555
Subset Accuracy:  0.9031285714285714
              precision    recall  f1-score   support

           0     0.7299    0.6739    0.7008      6718
           1     0.0000    0.0000    0.0000       631
           2     0.6726    0.8357    0.7453      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6014    0.7060    0.6496      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.6765    0.6558    0.6660     15611
   macro avg     0.3340    0.3693    0.3493     15611
weighted avg     0.6160    0.6558    0.6323     15611
 samples avg     0.0560    0.0583    0.0546     15611




Micro Accuracy:  0.9784404761904761
Subset Accuracy:  0.9118
              precision    recall  f1-score   support

           0     0.8158    0.6235    0.7068      6718
           1     0.0000    0.0000    0.0000       631
           2     0.7710    0.7533    0.7621      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6890    0.6587    0.6735      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.7671    0.6031    0.6753     15611
   macro avg     0.3793    0.3393    0.3571     15611
weighted avg     0.6971    0.6031    0.6445     15611
 samples avg     0.0537    0.0517    0.0504     15611




Micro Accuracy:  0.9762761904761905
Subset Accuracy:  0.9069142857142857
              precision    recall  f1-score   support

           0     0.7556    0.6884    0.7205      6718
           1     0.0000    0.0000    0.0000       631
           2     0.6710    0.8535    0.7513      3835
           3     0.0000    0.0000    0.0000       222
           4     0.5926    0.7666    0.6685      3548
           5     0.0000    0.0000    0.0000       657

   micro avg     0.6811    0.6802    0.6806     15611
   macro avg     0.3365    0.3848    0.3567     15611
weighted avg     0.6247    0.6802    0.6465     15611
 samples avg     0.0567    0.0607    0.0560     15611



...


Micro Accuracy:  0.9774904761904762
Subset Accuracy:  0.9061571428571429
              precision    recall  f1-score   support

           0     0.7242    0.7508    0.7373      6718
           1     0.4485    0.2900    0.3523       631
           2     0.6988    0.8501    0.7671      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6242    0.7847    0.6953      3548
           5     0.7269    0.2755    0.3996       657

   micro avg     0.6838    0.7336    0.7078     15611
   macro avg     0.5371    0.4918    0.4919     15611
weighted avg     0.6739    0.7336    0.6948     15611
 samples avg     0.0624    0.0668    0.0619     15611




Micro Accuracy:  0.9772904761904762
Subset Accuracy:  0.903
              precision    recall  f1-score   support

           0     0.6960    0.7663    0.7294      6718
           1     0.7179    0.0444    0.0836       631
           2     0.7239    0.8245    0.7709      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6211    0.7827    0.6926      3548
           5     0.6832    0.3349    0.4494       657

   micro avg     0.6830    0.7261    0.7039     15611
   macro avg     0.5737    0.4588    0.4543     15611
weighted avg     0.6763    0.7261    0.6830     15611
 samples avg     0.0647    0.0671    0.0632     15611




Micro Accuracy:  0.9790428571428571
Subset Accuracy:  0.9097428571428572
              precision    recall  f1-score   support

           0     0.7492    0.7270    0.7379      6718
           1     1.0000    0.0032    0.0063       631
           2     0.7825    0.7700    0.7762      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6927    0.6987    0.6957      3548
           5     0.5407    0.4749    0.5057       657

   micro avg     0.7356    0.6809    0.7072     15611
   macro avg     0.6275    0.4456    0.4536     15611
weighted avg     0.7352    0.6809    0.6879     15611
 samples avg     0.0631    0.0615    0.0597     15611




Micro Accuracy:  0.9768595238095238
Subset Accuracy:  0.9034571428571428
              precision    recall  f1-score   support

           0     0.7131    0.7612    0.7364      6718
           1     0.4343    0.2726    0.3350       631
           2     0.6846    0.8516    0.7590      3835
           3     0.0000    0.0000    0.0000       222
           4     0.6102    0.7914    0.6891      3548
           5     0.6610    0.3531    0.4603       657

   micro avg     0.6704    0.7426    0.7046     15611
   macro avg     0.5172    0.5050    0.4966     15611
weighted avg     0.6591    0.7426    0.6929     15611
 samples avg     0.0624    0.0678    0.0623     15611



KeyboardInterrupt: ignored